In [ ]:
import rasterio
import os
#import kelp_tools as kt
import pickle
import numpy as np
from rasterio.errors import RasterioIOError
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import clear_output


In [ ]:
def load_processed_img(path, file, cp=False, bands=None, just_data=False):
    try:
        if bands==None:
            with rasterio.open(os.path.join(path,file), 'r') as src:
                data = src.read()
                metadata = src.tags()
                transform = src.transform
                crs = src.crs
        else:
            with rasterio.open(os.path.join(path,file), 'r') as src:
                data = src.read(bands)
                metadata = src.tags()
                transform = src.transform
                crs = src.crs
    except RasterioIOError as e:
        print(f"Error reading file {file}: {e}")
        return None
    try:
        tide = float(metadata['TIDE'])
        current =float(metadata['CURRENT'])
    except:
        print(f'{file} has no TIDE or CURRENT metadata')
        return None
    if(just_data):
        return data, tide, current
    else:
        return data, metadata['TIMESTAMP'], transform, crs


def get_sensor(granule):
    file_data = granule.split('.')
    return file_data[1]

def view_rgb(rgb1, rgb2, kelp1, kelp2, isla_vista=False, title_1='rgb1', title_2='rgb2'):
    rgb_1 = np.stack([rgb1[2], rgb1[1], rgb1[0]], axis=-1)
    rgb_2 = np.stack([rgb2[2], rgb2[1], rgb2[0]], axis=-1)
    if(isla_vista):
        rgb_1 = rgb_1
        rgb_2 = rgb_2[2800:3200,800:1600]
        kelp1 = kelp1[2800:3200,800:1600]
        kelp2 = kelp2[2800:3200,800:1600]
    plt.figure(figsize=(50, 25)) 
    plt.subplot(2, 2, 1) 
    plt.imshow(rgb_1)
    plt.title(title_1)
    plt.subplot(2, 2, 2) 
    plt.imshow(rgb_2)
    plt.title(title_2)
    plt.subplot(2, 2, 3) 
    plt.imshow(kelp1)
    plt.title(title_1)
    plt.subplot(2, 2, 4) 
    plt.imshow(kelp2)
    plt.title(title_2)
    plt.show()
    

In [ ]:
path = r'H:\HLS_data\imagery\Isla_vista_kelp\processed_v2\11SKU' #'/mnt/h/Hls_data/imagery/Isla_vista_kelp/processed/11SKU'
#files = os.listdir(path)
object_path = r'C:\Users\attic\HLS_Kelp\python_objects'#'/mnt/c/Users/attic/hls_kelp/python_objects'
object_name = f'file_pairs_3_v2'
data = []
bands = [5,6]
with open(os.path.join(object_path,object_name), 'rb') as f:
    file_list = pickle.load(f)
for i , pair in enumerate(file_list):
    landsat = None
    sentinel = None
    files = pair[1]
    for img in files:
        sensor = get_sensor(img)
        if sensor == 'L30':
            landsat = img
        else:
            sentinel = img
    if landsat == None or sentinel == None:
        print('missing landsat or sentinel, skipping')
        continue
    l_data = load_processed_img(path,landsat, bands=bands, just_data=True)
    if l_data is None:
        continue
    l_img, l_tide, l_current = l_data
    s_data = load_processed_img(path,sentinel, bands=bands, just_data=True)
    if s_data is None:
        continue
    s_img, s_tide, s_current = s_data
    s_img = s_img[:,2800:3050,850:1600]
    l_img = l_img[:,2800:3050,850:1600]
    #if l_img == None:
    #    print(f'error loading {landsat}')
    #    continue
    
    l_mesma = np.array(l_img[1])
    l_mesma = np.where(l_mesma < 10, 0 , l_mesma)
    #l_mesma = np.where(l_mesma > 150, 0, l_mesma)
    l_mesma = np.where(s_img[0] == 2, 0, l_mesma)
    #l_mesma = np.where(s_img[0] == 0, l_mesma, 0)
    
    l_kelp = np.where(l_img[0] == 0, 1, 0)
    l_kelp = np.where(s_img[0] == 2, 0, l_kelp)
    l_kelp_pixels = np.sum(l_kelp)
    l_sum = np.sum(l_mesma)
    #print(l_sum)
 
    # if s_img ==None:
    #     print(f'error loading {sentinel}')
    #     continue
    s_mesma = np.array(s_img[1])
    s_mesma = np.where(s_mesma < 10, 0 , s_mesma)
    
    #s_mesma = np.where(s_mesma > 150, 0, s_mesma)
    s_mesma = np.where(l_img[0] == 2, 0, s_mesma)
    #s_mesma = np.where(l_img[0] == 0, s_mesma, 0)
    s_kelp = np.where(s_img[0] == 0, 1, 0)
    s_kelp = np.where(l_img[0] == 2, 0, s_kelp)
    s_kelp_pixels = np.sum(s_kelp)
    s_sum = np.sum(s_mesma)
    #print(s_sum)
    data.append([l_sum,s_sum,l_kelp_pixels,s_kelp_pixels, l_current, s_current, l_tide, s_tide, landsat, sentinel])

    clear_output()
    print(f'{i}/40')
    # if i > 60:
    #     break

data_array =np.stack(data)
column_keys = ['l_sum', 's_sum', 'l_kelp_pixels', 's_kelp_pixels', 'l_current', 's_current', 'l_tide', 's_tide', 'landsat', 'sentinel']
df = pd.DataFrame(data_array, columns=column_keys)

#print(data_array.shape)



In [ ]:
outlier_data = []
deviation_arr = []
#print(data_array)
for pair in data_array:
    num1 = int(pair[0])
    num2 = int(pair[1])
    if (num1 + num2) == 0:
        continue
    percent_deviation = abs(num1 - num2)/((num1 + num2)/2)
    deviation_arr.append(percent_deviation) 
    if percent_deviation > 1.7:
        outlier_data.append(pair)

outlier_data = np.stack(outlier_data)
#print(outlier_data)
mean = np.mean(percent_deviation)
print(mean)

In [ ]:
bands=[1,2,3,5]
for pair in data_array:
    print(f'Landsat Value: {pair[0]} | Sentinel 2 Value: {pair[1]}')
    lsat= load_processed_img(path,pair[4], bands=bands, just_data=True)
    s2 = load_processed_img(path,pair[5], bands=bands, just_data=True)
    print(pair[4],pair[5])
    view_rgb(lsat,s2,lsat[3],s2[3], isla_vista=True, title_1='landsat',title_2='sentinel2')

In [ ]:
df['l_tide'] = df['l_tide'].astype(float)
df['s_tide'] = df['s_tide'].astype(float)
df['s_sum'] = df['s_sum'].astype(int)
df['l_sum'] = df['l_sum'].astype(int)
tide_diff = df['l_tide'] - df['s_tide']

mesma_ht = np.where(tide_diff > 0, df['l_sum'], df['s_sum'])
mesma_lt = np.where(tide_diff <= 0, df['l_sum'], df['s_sum'])
tide_diff = abs(tide_diff)
mesma_diff = (mesma_lt - mesma_ht) / mesma_ht

plt.figure()
plt.scatter(tide_diff, mesma_diff, )
plt.ylim(0, 5)
plt.show()


In [ ]:
df['l_current'] = abs(df['l_current'].astype(float))
df['s_current'] = abs(df['s_current'].astype(float))
current_diff = df['l_current'] - df['s_current']

mesma_ht = np.where(current_diff > 0, df['l_sum'], df['s_sum'])
mesma_lt = np.where(current_diff <= 0, df['l_sum'], df['s_sum'])
current_diff = abs(tide_diff)
mesma_diff = (mesma_lt - mesma_ht) / mesma_ht

plt.figure()
plt.scatter(current_diff, mesma_diff, )
plt.ylim(0, 5)
plt.show()


In [ ]:
l_mesma = data_array[:, 0].astype(int)
l_kelp = data_array[:,2].astype(int)
s_mesma = data_array[:, 1].astype(int)
s_kelp = data_array[:,3].astype(int)

min_val = min(l_mesma.min(), s_mesma.min())
max_val = max(s_mesma.max(), s_mesma.max())
x = np.linspace(min_val, max_val, 100)
y = x
slope, intercept = np.polyfit(l_mesma, s_mesma, 1)
print(slope, intercept)
y_fit = slope * x + intercept
plt.figure()
plt.scatter(l_mesma, s_mesma)
plt.plot(x, y, color='red', label='y = x')
#plt.plot(x, y_fit, color='blue', label='y = x')
# Labels and title (optional)
plt.xlabel('Landsat')
plt.ylabel('Seninel 2')
plt.title('Mesma Pixel Summation Comparison')
plt.show()

plt.figure()
plt.scatter(l_kelp, s_kelp)
#plt.plot(x, y, color='red', label='y = x')
# Labels and title (optional)
plt.xlabel('Landsat')
plt.ylabel('Seninel 2')
plt.title('Classified Pixel Count Comparison')
plt.show()